In [6]:
import pandas as pd
import nltk #procesamiento de palabras
from nltk.corpus import stopwords
import gensim #pip install gensim sirve separa texto 
import re #sirve para concatenar texto


from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator #pip install wordcloud
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import warnings


In [23]:
filename ='C:/Users/F32740/OneDrive - Kimberly-Clark/Documents/U/Análisis de datos/Tarea RFM/RFM.xlsx'
#filename = '/Users/cmoralesal/Documents/Carlos Morales/BAC-comentarios/clasificacion_1.xlsx'
df = pd.read_excel(filename)
df.head(5)

,ID_CLIENTE,VISITAS,MONTO_DOLARES,FACTURA
0,CG-12520,2016-11-08,261.9600,1000
1,CG-12520,2016-11-08,731.9400,1000
2,DV-13045,2016-06-12,14.6200,1001
3,SO-20335,2015-10-11,957.5775,1002
4,SO-20335,2015-10-11,22.3680,1002


In [10]:
df.tail()

,ID_CLIENTE,VISITAS,MONTO_DOLARES
9989,TB-21400,2014-01-21,25.248
9990,DB-13060,2017-02-26,91.960
9991,DB-13060,2017-02-26,258.576
9992,DB-13060,2017-02-26,29.600
9993,CC-12220,2017-05-04,243.160


In [11]:
df.describe()

,MONTO_DOLARES
count,9994.000000
mean,229.858001
std,623.245101
min,0.444000
25%,17.280000
50%,54.490000
75%,209.940000
max,22638.480000


In [12]:
import datetime as dt

In [13]:
NOW = dt.date(2021,11,20) 

In [15]:
df['date'] = pd.DatetimeIndex(df.VISITAS).date

In [16]:
df.head()

,ID_CLIENTE,VISITAS,MONTO_DOLARES,date
0,CG-12520,2016-11-08,261.9600,2016-11-08
1,CG-12520,2016-11-08,731.9400,2016-11-08
2,DV-13045,2016-06-12,14.6200,2016-06-12
3,SO-20335,2015-10-11,957.5775,2015-10-11
4,SO-20335,2015-10-11,22.3680,2015-10-11


In [17]:
df_recency = df.groupby(['ID_CLIENTE'],as_index=False)['date'].max()
df_recency.columns = ['ID_CLIENTE','Last_Purchase_Date']

In [18]:
df_recency['Recency'] = df_recency.Last_Purchase_Date.apply(lambda x:(NOW - x).days)

In [19]:
df_recency.head()

,ID_CLIENTE,Last_Purchase_Date,Recency
0,AA-10315,2017-06-29,1605
1,AA-10375,2017-12-11,1440
2,AA-10480,2017-04-15,1680
3,AA-10645,2017-11-05,1476
4,AB-10015,2016-11-10,1836


In [20]:
df_recency.drop(columns=['Last_Purchase_Date'],inplace=True)

In [25]:
FM_Table = df.groupby('ID_CLIENTE').agg({'FACTURA'   : lambda x:len(x),
                                         'MONTO_DOLARES'  : lambda x:x.sum()})

In [26]:
FM_Table.rename(columns = {'FACTURA' :'Frequency',
                           'MONTO_DOLARES':'Monetary_Value'},inplace= True) 

In [27]:
FM_Table.head()

,Frequency,Monetary_Value
ID_CLIENTE,,
AA-10315,11,5563.560
AA-10375,15,1056.390
AA-10480,12,1790.512
AA-10645,18,5086.935
AB-10015,6,886.156


In [28]:
RFM_Table = df_recency.merge(FM_Table,left_on='ID_CLIENTE',right_on='ID_CLIENTE')
RFM_Table.head()

,ID_CLIENTE,Recency,Frequency,Monetary_Value
0,AA-10315,1605,11,5563.560
1,AA-10375,1440,15,1056.390
2,AA-10480,1680,12,1790.512
3,AA-10645,1476,18,5086.935
4,AB-10015,1836,6,886.156


In [38]:
df[df.ID_CLIENTE == 'AA-10315']

,ID_CLIENTE,VISITAS,MONTO_DOLARES,FACTURA
1159,AA-10315,2017-06-29,362.940,1397
1160,AA-10315,2017-06-29,11.540,1397
1299,AA-10315,2015-10-04,26.960,1397
2229,AA-10315,2014-03-31,673.568,1397
2230,AA-10315,2014-03-31,52.980,1397
5198,AA-10315,2016-03-03,3930.072,1397
5199,AA-10315,2016-03-03,2.304,1397
5200,AA-10315,2016-03-03,431.976,1397
5201,AA-10315,2016-03-03,41.720,1397
7468,AA-10315,2014-09-15,14.940,1397


In [37]:
RFM_Table[RFM_Table.ID_CLIENTE == 'AA-10315']

,ID_CLIENTE,Recency,Frequency,Monetary_Value
0,AA-10315,1605,11,5563.56


In [43]:
(NOW - dt.date(2017,6,29)).days == 325

False

In [44]:
quantiles = RFM_Table.quantile(q=[0.25,0.50,0.75])
quantiles = quantiles.to_dict()

In [55]:
quantiles

{'Recency': {0.25: 1451.0, 0.5: 1496.0, 0.75: 1604.0},
 'Frequency': {0.25: 8.0, 0.5: 12.0, 0.75: 16.0},
 'Monetary_Value': {0.25: 1146.0500000000002, 0.5: 2256.394, 0.75: 3785.276}}

In [45]:
segmented_rfm = RFM_Table.copy()

In [46]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [47]:
segmented_rfm['R_quartile'] = segmented_rfm['Recency'].apply(RScore, args=('Recency',quantiles))
segmented_rfm['F_quartile'] = segmented_rfm['Frequency'].apply(FMScore, args=('Frequency',quantiles))
segmented_rfm['M_quartile'] = segmented_rfm['Monetary_Value'].apply(FMScore, args=('Monetary_Value',quantiles))
segmented_rfm.head()

,ID_CLIENTE,Recency,Frequency,Monetary_Value,R_quartile,F_quartile,M_quartile
0,AA-10315,1605,11,5563.560,4,3,1
1,AA-10375,1440,15,1056.390,1,2,4
2,AA-10480,1680,12,1790.512,4,3,3
3,AA-10645,1476,18,5086.935,2,1,1
4,AB-10015,1836,6,886.156,4,4,4


Best Recency score = 1: Purchased very recently. Best Frequency score = 1: highest quantity purchases. Best Monetary score = 1: spent the most.

In [48]:
segmented_rfm['RFM_Segment'] = segmented_rfm.R_quartile.map(str)+segmented_rfm.F_quartile.map(str)+segmented_rfm.M_quartile.map(str)
segmented_rfm.head()

,ID_CLIENTE,Recency,Frequency,Monetary_Value,R_quartile,F_quartile,M_quartile,RFM_Segment
0,AA-10315,1605,11,5563.560,4,3,1,431
1,AA-10375,1440,15,1056.390,1,2,4,124
2,AA-10480,1680,12,1790.512,4,3,3,433
3,AA-10645,1476,18,5086.935,2,1,1,211
4,AB-10015,1836,6,886.156,4,4,4,444


In [49]:
segmented_rfm['RFM_Score'] = segmented_rfm[['R_quartile','F_quartile','M_quartile']].sum(axis=1)
segmented_rfm.head()

,ID_CLIENTE,Recency,Frequency,Monetary_Value,R_quartile,F_quartile,M_quartile,RFM_Segment,RFM_Score
0,AA-10315,1605,11,5563.560,4,3,1,431,8
1,AA-10375,1440,15,1056.390,1,2,4,124,7
2,AA-10480,1680,12,1790.512,4,3,3,433,10
3,AA-10645,1476,18,5086.935,2,1,1,211,4
4,AB-10015,1836,6,886.156,4,4,4,444,12


In [50]:
print("Best Customers: ",len(segmented_rfm[segmented_rfm['RFM_Segment']=='111']))
print('Loyal Customers: ',len(segmented_rfm[segmented_rfm['F_quartile']==1]))
print("Big Spenders: ",len(segmented_rfm[segmented_rfm['M_quartile']==1]))
print('Almost Lost: ', len(segmented_rfm[segmented_rfm['RFM_Segment']=='134']))
print('Lost Customers: ',len(segmented_rfm[segmented_rfm['RFM_Segment']=='344']))
print('Lost Cheap Customers: ',len(segmented_rfm[segmented_rfm['RFM_Segment']=='444']))

Best Customers:  29
Loyal Customers:  188
Big Spenders:  198
Almost Lost:  16
Lost Customers:  30
Lost Cheap Customers:  58


Suggested marketing strategies on segmented customers:
Best Customers- No price incentives, new products, and loyalty programs.
Big Spenders- Market your most expensive products.
Almost Lost- Aggresive price incentives
Lost Customers-Don’t spend too much trying to re-acquire them.

In [51]:
segmented_rfm['RFM_Score'].unique()

array([ 8,  7, 10,  4, 12,  9,  5, 11,  6,  3], dtype=int64)

In [52]:
segmented_rfm.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary_Value': ['mean', 'count'] }).round(1)

Recency Frequency Monetary_Value      
             mean      mean           mean count
RFM_Score                                       
3          1437.5      22.8         6164.3    29
4          1448.9      19.4         4955.6    79
5          1476.5      18.1         4525.1    92
6          1525.7      16.0         4556.0   100
7          1523.9      12.5         2929.7    89
8          1549.4      11.0         2541.0   100
9          1558.2       9.6         1453.7   102
10         1620.2       7.9         1272.5    80
11         1712.3       6.4          956.4    64
12         1896.4       5.0          498.5    58